Bài toán: Sau khi bitcoin và các đồng tiền ảo sụt giảm, công ty có nhu cầu khảo sát thông tin về card đồ họa thông qua một website. Yêu cầu team Data Engineer collect và thống kê các thông tin tại danh mục trên website: [https://www.newegg.com](https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48?Tid=7709) để phía đội kinh doanh có cơ sở triển khai chiến dịch mới.

Mô tả cụ thể:

- Nguồn dữ liệu cần crawl là danh mục sau: https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48?Tid=7709
- Thông tin cần lấy:
    - ItemID
    - Title
    - Branding (hang)
    - Rating
    - So luong rating
    - Price (Current Price) --> Chuyen doi duoi dang number
    - Shipping (Free, ko ship hay mat phi)
    - Image URL 
    - Cac thong tin chi tiet ve san pham: 
        - MaxResolution
        - DisplayPort
        - HDMI
        - DirectX
        - Model
- Số lượng: Toàn bộ các sản phẩm của 100 pages (khoảng 3600 sản phẩm)
- Thông tin sau khi collect đẩy vào một cơ sở dữ liệu MySQL
    - Thêm một cột total price dựa trên giá shipping 
    - Thông tin chi tiết về sản phẩm lưu dưới dạng JSON

In [1]:
# Import Libraries

import requests
import webbrowser
import re
import pandas as pd
import json
from bs4 import BeautifulSoup

def web_scraping():
    
    ls_item_id = []
    ls_title = []
    ls_brand = []
    ls_rating = []
    ls_number_of_rating = []
    ls_price_current = []
    ls_shipping = []
    ls_image_url = []
    ls_details = []
    
    for i in range(1, 101):
        if i == 1:
            url = "https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48"
        else:
            url = "https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48/Page-" + str(i)

        # Fetching content of one page    
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')

        # Filter items cell 
        items = soup.findAll('div', attrs={'class': re.compile('item-container')})

        # Extract information

        for item in items:

            # Get item_id
            item_id = item['id']
            ls_item_id.append(item_id)

            # Get title
            try:
                title = item.find('a').find('img')['title']
                ls_title.append(title)
            except:
                title = 'Not title'
                ls_title.append(title)
                
            # Get branding
            try:
                brand = item.find('a', attrs={'class': re.compile('item-brand')}).find('img')['title']
                ls_brand.append(brand)
            except: 
                brand = 'Not brand'
                ls_brand.append(brand)
                
            # Get rating
            try:
                rating = item.find('i', attrs={'class': re.compile("rating")})['aria-label'].split()[1]
                ls_rating.append(rating)
            except:
                rating = 'not rating'
                ls_rating.append(rating)

            # Get number of rating
            try:
                number_of_rating = item.find("span", attrs={'class': 'item-rating-num'}).string.strip("()")
                ls_number_of_rating.append(number_of_rating)
            except:
                number_of_rating = 0
                ls_number_of_rating.append(number_of_rating)

            # Get price current and convert to number
            try:
                price_current = item.find('li', attrs={'class': 'price-current'}).find(['strong']).string + \
                            items[0].find('li', attrs={'class': 'price-current'}).find(['sup']).string

                price_current = float(price_current.replace(',',''))

                ls_price_current.append(price_current)
                
            except:
                price_current = 0
                ls_price_current.append(price_current)
                
            # Get Shipping
            try:
                shipping = item.find('li', attrs={'class': 'price-ship'}).string
                ls_shipping.append(shipping)
            except:
                shipping = 'Not information about shipping'
                ls_shipping.append(shipping)
                
            # Get image URL
            try:
                image_url = item.find('img')['src']
                ls_image_url.append(image_url)
            except:
                image_url = 'Not information about image url'
                ls_image_url.append(image_url)
                
            # Get details of product
            dict_details = {}
            try:
                for d in item.findAll('li'):
                    if d.text.startswith('Max Resolution'):
                        dict_details["Max_Resolution"] = d.text.split(":")[1]
                    elif d.text.startswith('DisplayPort'):
                        dict_details["Display_Port"] = d.text.split(":")[1]
                    elif d.text.startswith('HDMI'):
                        dict_details["HDMI"] = d.text.split(":")[1]
                    elif d.text.startswith('DirectX'):
                        dict_details["DirectX"] = d.text.split(":")[1]
                    elif d.text.startswith('Model'):
                        dict_details["Model"] = d.text.split(":")[1]
                    else:
                        continue
            except:
                pass

            ls_details.append(dict_details)
            
    return ls_item_id, ls_title, ls_brand, ls_rating, ls_number_of_rating, \
           ls_price_current, ls_shipping, ls_image_url, ls_details

In [2]:
# Invoke function scraping web

ls_item_id, ls_title, ls_brand, ls_rating, ls_number_of_rating, \
ls_price_current, ls_shipping, ls_image_url, ls_details = web_scraping()

In [48]:
# Create Dataframe from lists

df = pd.DataFrame(list(zip(ls_item_id, ls_title, ls_brand, ls_rating, ls_number_of_rating, \
                           ls_price_current, ls_shipping, ls_image_url, ls_details)),
                 columns=['item_id','title','brand','rating','number_of_rating','price_current','shipping', \
                          'image_url', 'details'])

# Convert columns details to str

df["details"] = df["details"].astype('str')

# Calculate price shipping 

df['price_shipping'] = df['shipping'].apply(lambda x: float(x.rstrip(" Shipping").lstrip("$")) 
                                            if x != 'Free Shipping' and x != 'Special Shipping' else 0)

# Create total price

df['total_price'] = df['price_current'] + df['price_shipping']

# Drop columns price shipping

df.drop(columns='price_shipping', inplace=True)

# Describe dataframe

print('Number of records: ',len(df), '\n')

print(df.dtypes)


Number of records:  3604 

item_id              object
title                object
brand                object
rating               object
number_of_rating     object
price_current       float64
shipping             object
image_url            object
details              object
total_price         float64
dtype: object


In [50]:
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="db_project2"
uname="tanlee"
pwd="17012021*Th"


# Create SQLAlchemy engine to connect to MySQL Database

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}" \
                       .format(host=hostname, db=dbname, user=uname, pw=pwd))

# Write dataframe to mysql database

df.to_sql(name='products', if_exists='replace', con=engine, index=False)

3604